## Geometric View of Probability Theory

__Random variable (function)__: given an experiment with sample space $S$, a random variable function is a function from the sample space $S$ to the real numbers $R$. It is common, but not required, to denote random variables by capital letters. 

One way to think about probability is as a new number concept that allows you to tackle problems that have a special kind of uncertainty built into them. Thus, the key idea is that there is some number, say $x$, with a traveling companion, say, $f(x)$, and this companion represents the uncertainties about the value of x as if looking at the number x through a frosted window. The degree of opacity of the window is represented by $f(x)$. If we want to manipulate x, then we have to figure out what to do with f (x). For example if we want y = 2x, then we have to understand how $f(x)$ generates $f(y)$.

Let us consider the following piecewise funtion:

$f(x) = \begin{cases}
1 & 0 < x \leq 1 \\
2 & 1 < x \leq 2 \\
0 & \ \text{otherwise} \end{cases}$


To compute the integral, we do:
$$\int_0^2 f d \mu = 1 \mu (\{(0, 1]\}) + 2 \mu (\{(1, 2]\})$$

By introducing the $\mu$ function as a way of measuring the intervals above, we have introduced another degree of freedom in our integration. 

Unfortunately, the term random variable is not very descriptive. A better term is __measurable function__. The measurable function maps a set into a number on the real line. For example, $\{1\} \to  1$ is one such function.

Let's consider a slightly more interesting problem where we toss two dice. We assume that each throw is independent, meaning that the outcome of one does not influence the other. What are the sets in this case? They are all pairs of possible outcomes from two throws as shown below,
$$\Omega = \{ (1, 1), (1, 2), \cdots, (5, 6), (6, 6) \}$$
What are the measures of each of these sets?  By virtue of the independence claim, the measure of each is the product of the respective measures of each element. For instance,
$$P((1, 2)) = P(\{1\}) P(\{2\}) = \frac{1}{6^2}$$

In [23]:
from collections import defaultdict
import numpy as np
import pandas as pd
import sympy as S
from sympy import stats, integrate, simplify
from sympy.stats import density, E, Die
from sympy.abc import y, x

In [16]:
# probability of sums of two dices
d = {(i, j): i+j for i in range(1, 7) for j in range(1, 7)}
dinv = defaultdict(list)
for i, j in d.items():
    dinv[j].append(i)
X = {i:np.round(len(j)/36.0, 3) for i, j in dinv.items()}
print(X)

# probability that half the product of three dice will exceed their sum
d = {(i, j, k):((i*j*k)/2 > i+j+k) for i in range(1, 7)
                                   for j in range(1, 7)
                                   for k in range(1, 7)}
dinv = defaultdict(list)
for i, j in d.items():
    dinv[j].append(i)
X={i:np.round(len(j)/6.0**3, 3) for i,j in dinv.items()}
print(X)  # approximately 0.63

# unfair dice
df = pd.DataFrame(index=[(i, j) for i in range(1, 7) for j in range(1, 7)],
            columns=['sm', 'd1', 'd2', 'pd1', 'pd2', 'p'])
df.d1 = [i[0] for i in df.index]
df.d2 = [i[1] for i in df.index]
df.sm = list(map(sum, df.index))
df.loc[df.d1<=3, 'pd1'] = 1/9
df.loc[df.d1>3, 'pd1'] = 2/9
df.pd2 = 1/6
df.p = df.pd1 * df.pd2
df.groupby('sm')['p'].sum()

{2: 0.028, 3: 0.056, 4: 0.083, 5: 0.111, 6: 0.139, 7: 0.167, 8: 0.139, 9: 0.111, 10: 0.083, 11: 0.056, 12: 0.028}
{False: 0.37, True: 0.63}


sm
2     0.018519
3     0.037037
4     0.055556
5     0.092593
6     0.129630
7     0.166667
8     0.148148
9     0.129630
10    0.111111
11    0.074074
12    0.037037
Name: p, dtype: float64

In [24]:
fxy = x + y                 # joint density
fy = integrate(fxy,(x,0,1)) # marginal density
fx = integrate(fxy,(y,0,1)) # marginal density
EXY = (3*y+2)/(6*y+3) # conditional expectation
LHS=integrate((x-EXY)**2*fxy,(x,0,1),(y,0,1)) 
print(LHS) # left-hand-side
# using Pythagorean theorem
RHS=integrate((x)**2*fx,(x,0,1))-integrate((EXY)**2*fy,(y,0,1))
RHS # right-hand-side
print(RHS)

-log(3)/144 + 1/12
-log(3)/144 + 1/12


Let $v = [1, 1]^T$ be the subspace onto which we want to _project_ $y$. At the closest point, the vector between $y$ and $x$ is perpendicular to the line. This means that
$$(y-x)^T v = 0 $$,
where $x$ is an arbitary point along the line:
$$x = \alpha v$$
We can derive the projection operator:
$$P_v = \frac{v v^T}{||v||^2}$$
Then, we can take any $y$ and find the closest point on $v$ by doing
$$P_v y = v ( \frac{v^T y}{||v||^2} ) $$

With the understanding of projection, we now define the inner product for the random variables $X$ and $Y$ as
$$<X, Y> = E(XY)$$
So, when we have the __orthogonal projection__, we would have
$$<X - h_{opt}(Y), Y> = 0$$

> One could study Fourier Analysis to understand tht the entire theory of probability could be built upon the concept of orthgonality.

__Conditional Expectation as Projection__. The conditional expectation is the minimum mean squared error (MMSE) solution to the following problem:
$$\min \int_{R} (x - h(y))^2 d x$$
with the minimizing $h_{opt}(y)$ as
$$h_{opt}(Y) = E(X|Y)$$

Suppose we have two fair six-sided dice ($X$ and $Y$) and we want to measure the sum of the two variables as $Z = X + Y$ . Further, let’s suppose that given $Z$, we want the best estimate of $X$ in the mean-squared-sense. Thus, we want to minimize the following:
$$J(\alpha) = \sum(x - \alpha z)^2 P(x, y)$$
We can substitute in for Z in J and get:
$$J(\alpha) = \sum(x - \alpha (x +y))^2 P(x, y)$$

In [21]:
# conditional expectation as projection
x = Die('D1', 6)  # 1st six sided die
y = Die('D2', 6)
a = S.symbols('a')
z = x + y
J = E((x-a*(x+y))**2)
print(S.simplify(J))
sol, = S.solve(S.diff(J, a), a)
print(sol)

329*a**2/6 - 329*a/6 + 91/6
1/2



Three coins, 10, 20 and 50p are tossed. The values of the coins that land heads up are totaled. What is the expected total given that two coins have landed heads up? In this case we have we want to compute $E(\xi |\eta)$ where
$$\xi := 10X_{10} + 20X_{20} + 50X_{50}$$
where $X_i \in \{0, 1\}$ and where $X_{10}$ is the Bernoulli-distributed random variable cor- responding to the 10p coin (and so on). Thus, $\xi$ represents the total value of the heads-up coins. The $\eta$ represents the condition that only two of the three coins are heads-up,
$$\eta := X_{10} X_{20}(1 − X_{50}) + (1 − X_{10})X_{20} X_{50} + X_{10}(1 − X_{20})X_{50}$$
and is a function that is non-zero only when two of the three coins lands heads-up.

To compute the conditional expectation, we want to find a function h of η that minimizes the mean-squared-error (MSE),
$$MSE = \sum_{X \in [0, 1]^3} \frac{1}{2^3}(\xi - h(\eta))^2$$

In [22]:
X10, X20, X50 = S.symbols('X10, X20, X50', real=True)
xi = 10*X10 + 20*X20 + 50*X50
eta = X10*X20*(1-X50) + X10*(1-X20)*(X50) + (1-X10)*X20*(X50)
num = S.summation(xi*eta, (X10, 0, 1), (X20, 0, 1), (X50, 0, 1))
den = S.summation(eta*eta, (X10, 0, 1), (X20, 0, 1), (X50, 0, 1))
alpha = num/den
print(alpha)

160/3


The insights from the above analysis is that given the additional measuremnt, we work with the conditional or a _posterior density_ $f_{Y|X}(y|x)$, and the best estimation is the conditional expectation:
$$\hat{y}(x) = E[Y |X = x]$$

### References

Lecture Notes from [MIT](https://ocw.mit.edu/courses/electrical-engineering-and-computer-science/6-011-introduction-to-communication-control-and-signal-processing-spring-2010/readings/MIT6_011S10_chap08.pdf)